<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---


<p align="center"><h1 align="center">Diamond Price Prediction Competition </h1>

---

Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data  with Sklearn Column Transformer/ Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [4]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/diamonds_competition_data-repository:latest') 


Data downloaded successfully.


In [5]:
# Separate data into X_train, y_train, and X_test
import pandas as pd

X_train = pd.read_csv("diamonds_competition_data/X_train.csv")
y_train = pd.read_csv("diamonds_competition_data/y_train.csv", squeeze = True)

X_test=pd.read_csv("diamonds_competition_data/X_test.csv")

X_train.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,0.90,Good,E,SI2,64.1,57.0,6.07,6.09,3.90
1,1.54,Premium,G,VS2,61.1,59.0,7.44,7.40,4.53
2,0.52,Ideal,D,SI1,62.4,56.0,5.17,5.12,3.21
3,0.85,Ideal,E,SI1,63.0,56.0,6.00,6.04,3.79
4,0.35,Ideal,D,VS2,60.9,57.0,4.55,4.51,2.76


##2.   Preprocess data using Sklearn Column Transformer / Write and Save Preprocessor function


In [6]:
# Simple Preprocessor with sklearn 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

numeric_features = ['carat', 'depth', 'table', 'x', 'y', 'z']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), 
    ('scaler', StandardScaler())])

categorical_features = ['cut', 'color', 'clarity']

# Replacing missing values with Modal value and then one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Final preprocessor object set up with ColumnTransformer...

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# fit preprocessor to your data
preprocess = preprocess.fit(X_train)

In [7]:
# Write function to transform data with preprocessor 
# In this case we use sklearn's Column transformer in our preprocessor function

def preprocessor(data):
    preprocessed_data=preprocess.transform(data)
    return preprocessed_data

In [8]:
# check shape of X data 
preprocessor(X_train).shape

(43152, 26)

##3. Fit model on preprocessed data and save preprocessor function and model 


In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

model = RandomForestRegressor(n_estimators = 500, max_depth = 3, random_state=0)
model.fit(preprocessor(X_train), y_train) # Fitting to the training set.
model_predictions = model.predict(preprocessor(X_train))
r2_score(model_predictions, y_train)

0.8570772067210946

#### Save preprocessor function to local "preprocessor.zip" file

In [10]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [11]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType

feature_count=preprocessor(X_test).shape[1]
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of features in preprocessed data

onnx_model = model_to_onnx(model, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=False)


with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [12]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://csyks5i22a.execute-api.us-east-1.amazonaws.com/prod/m" #This is the unique rest api that powers this Diamond Price Prediction Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [13]:
#Instantiate Competition
import aimodelshare as ai
mycompetition= ai.Competition(apiurl)

In [14]:
#Submit Model 1: 

#-- Generate predicted values (a list of predicted diamond prices) (Model 1)
predicted_values = model.predict(preprocessor(X_test))

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=predicted_values)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 1

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1747


In [15]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,mse,rmse,mae,r2,ml_framework,transfer_learning,deep_learning,model_type,username,version
0,1895117.90,1376.63,777.73,0.88,sklearn,False,False,RandomForestRegressor,ML_Risk_Mgmnt,1


## 5. Repeat submission process to improve place on leaderboard


In [16]:
# Create model 2 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

model_2 = RandomForestRegressor(n_estimators = 300, max_depth = 2, random_state=0)
model_2.fit(preprocessor(X_train), y_train) # Fitting to the training set.
model_predictions = model_2.predict(preprocessor(X_train))
r2_score(model_predictions, y_train) 

0.7944122317100049

In [17]:
# Save Model 2 to .onnx file

# Check how many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType

feature_count=preprocessor(X_test).shape[1]
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of features in preprocessed data

onnx_model = model_to_onnx(model_2, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=False)

# Save model to local .onnx file
with open("model_2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString()) 

In [18]:
# Submit Model 2

#-- Generate predicted y values (Model 2)
prediction_labels = model_2.predict(preprocessor(X_test))

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model_2.onnx",
                                 prediction_submission=prediction_labels,
                                 preprocessor_filepath="preprocessor.zip")

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 2

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1747


In [19]:
# Compare two or more models
data=mycompetition.compare_models([1,2], verbose=1)
mycompetition.stylize_compare(data)

,param_name,default_value,model_version_1,model_version_2
0,bootstrap,True,True,True
1,ccp_alpha,0.000000,0.000000,0.000000
2,criterion,mse,mse,mse
3,max_depth,None,3,2
4,max_features,auto,auto,auto
5,max_leaf_nodes,None,None,None
6,max_samples,None,None,None
7,min_impurity_decrease,0.000000,0.000000,0.000000
8,min_impurity_split,None,None,None
9,min_samples_leaf,1,1,1


In [22]:
# Submit a third model using GridSearchCV

from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {'n_estimators': np.arange(10, 50, 100),'max_depth':[1, 3]} #np.arange creates sequence of numbers for each k value

gridmodel = GridSearchCV(RandomForestRegressor(), param_grid=param_grid, cv=10)

#use meta model methods to fit score and predict model:
gridmodel.fit(preprocessor(X_train), y_train)

#extract best score and parameter by calling objects "best_score_" and "best_params_"
print("best mean cross-validation score: {:.3f}".format(gridmodel.best_score_))
print("best parameters: {}".format(gridmodel.best_params_))


best mean cross-validation score: 0.875
best parameters: {'max_depth': 3, 'n_estimators': 10}


In [23]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

feature_count=preprocessor(X_test).shape[1] #Get count of preprocessed features
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of preprocessed features

onnx_model = model_to_onnx(gridmodel, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=False)

with open("gridmodel.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [24]:
#Submit Model 3: 

#-- Generate predicted values
prediction_labels = gridmodel.predict(preprocessor(X_test))

# Submit Model 3 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "gridmodel.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 3

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1747


In [25]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,mse,rmse,mae,r2,ml_framework,transfer_learning,deep_learning,model_type,username,version
0,1865445.56,1365.81,773.38,0.88,sklearn,False,False,RandomForestRegressor,ML_Risk_Mgmnt,3
1,1895117.90,1376.63,777.73,0.88,sklearn,False,False,RandomForestRegressor,ML_Risk_Mgmnt,1
2,2588887.91,1609.00,1006.14,0.84,sklearn,False,False,RandomForestRegressor,ML_Risk_Mgmnt,2


In [27]:
# Compare two or more models
data=mycompetition.compare_models([1, 2, 3], verbose=1)
mycompetition.stylize_compare(data)

,param_name,default_value,model_version_1,model_version_2,model_version_3
0,bootstrap,True,True,True,True
1,ccp_alpha,0.000000,0.000000,0.000000,0.000000
2,criterion,mse,mse,mse,mse
3,max_depth,None,3,2,3
4,max_features,auto,auto,auto,auto
5,max_leaf_nodes,None,None,None,None
6,max_samples,None,None,None,None
7,min_impurity_decrease,0.000000,0.000000,0.000000,0.000000
8,min_impurity_split,None,None,None,None
9,min_samples_leaf,1,1,1,1


In [31]:
# Here are several classic ML architectures you can choose from to experiment with next:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor


#Example code to fit model:
model = GradientBoostingRegressor(n_estimators=50, learning_rate=.1,
    max_depth=1, random_state=0).fit(preprocessor(X_train), y_train)
model.score(preprocessor(X_train), y_train)

# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

feature_count=preprocessor(X_test).shape[1] #Get count of preprocessed features
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of preprocessed features

onnx_model = model_to_onnx(model, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=False)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

#-- Generate predicted values (a list of predicted labels "real" or "fake")
prediction_labels = model.predict(preprocessor(X_test))

# Submit model to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)


Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 4

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1747
